In [6]:
import json
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the NLLB model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B", device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B", device_map="auto")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 1; 23.64 GiB total capacity; 17.69 GiB already allocated; 73.06 MiB free; 17.71 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
count = 0
total_data_length = 0

def translate_to_target_language(target_language_code):
    global count
    global total_data_length
    # Recursively process each item in the JSON object
    def process_item(item):
        if isinstance(item, dict):
            new_translations = {}  # Temporary dictionary to store new translations
            for key, value in item.items():
                if key.endswith("FR"):
                    # Get the corresponding English key (without the -FR suffix)
                    english_key = key[:-3]
                    if english_key in item:
                        english_text = item[english_key]
                        # Translate the English text to the target language
                        inputs = tokenizer(english_text, return_tensors="pt")
                        translated_tokens = model.generate(
                            **inputs, forced_bos_token_id=tokenizer.lang_code_to_id[f"{target_language_code.split('_')[0]}_Latn"], max_length=30
                        )
                        target_translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
                        # Create the new key for the translated text
                        new_key = f"{english_key}_{target_language_code}"
                        new_translations[new_key] = target_translation  # Store new translation in temporary dictionary
                        # print(f"Translated {english_key} to {new_key}: {target_translation}")
                    else:
                        # # Issue a warning if there are any other sibling tags without an underscore
                        # for sibling_key in item.keys():
                        #     if '_' not in sibling_key and sibling_key != key:
                        #         print(f'Warning: Skipping translation for {key} due to sibling tag {sibling_key} without an underscore.')
                        #         break
                        continue

                print(f"Processing {count} of {total_data_length} items")
                count += 1
                
                # Recursively process nested items
                process_item(value)

            item.update(new_translations)  # Update original dictionary with new translations

        elif isinstance(item, list):
            for value in item:
                process_item(value)

    
    # Load the JSON file
    with open("HOTTP.json", "r", encoding="utf-8") as file:
        data = json.load(file)
        total_data_length = len(data)
        print(f"Loaded {total_data_length} items from HOTTP.json")

    # Process the loaded data
    process_item(data)

    # Save the updated JSON data to a new file
    with open(f"HOTTP_translated_{target_language_code}.json", "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

# Translate to a specific target language (e.g., 'spa_Latn')
translate_to_target_language('spa_Latn')


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!